# Import Libraries

In [1]:
from swarmintelligence import ArchimedesOptimizationAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))
rgb_misc_dataset = dict(itertools.islice(rgb_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
# optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
# optimal_params

In [5]:
optimal_params = {
    'objectSize':30,
    'maxIteration': 10
}

# Evaluation

In [6]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
# n_runs = 30
n_runs = 1

In [7]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = ArchimedesOptimizationAlgorithm(
                        k=threshold,
                        objectSize=optimal_params['objectSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = ArchimedesOptimizationAlgorithm(
                        k=threshold,
                        objectSize=optimal_params['objectSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = ArchimedesOptimizationAlgorithm(
                        k=threshold,
                        objectSize=optimal_params['objectSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
[ 85 183]
[ 63 109]
[202 195]
927.3412154361523  |  2428.288083881008  |  1787.8045911666616
Mean Fitness = 1714.4779634946071
	 AirplaneF16.tiff | Threshold =  3
[149 223 147]
[158 185  79]
[106 127  48]
956.660425684024  |  2531.457043226552  |  1846.3872798698833
Mean Fitness = 1778.1682495934863
	 AirplaneF16.tiff | Threshold =  4
[163  27 219  25]
[109  53 104 212]
[ 57 128 164  58]
969.7220766629255  |  2518.0552944227993  |  1882.229683035869
Mean Fitness = 1790.0023513738645
	 AirplaneF16.tiff | Threshold =  5
[120 188 135  82  93]
[222 155 158 125 220]
[130 134 188 156 148]
1002.5588096739694  |  2598.0194853715097  |  1910.5258458694248
Mean Fitness = 1837.0347136383014
	 Lena.png | Threshold =  2
[198 179]
[221 208]
[161 198]
1003.6570185970401  |  2393.0804857472363  |  2155.6390194411747
Mean Fitness = 1850.7921745951505
	 Lena.png | Threshold =  3
[ 21 197 203]
[140 160 208]
[175 246 187]
1067.482394152646  |  2587.8805202304575  |  225

## Visualize Results using DataFrame

In [8]:
gwo_evaluation_results_df = pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[120, 178]","[108, 169]","[115, 182]",927.341215,2428.288084,1787.804591,...,"[[115, 255, 255, 255, 255, 255, 255, 255, 255,...",[927.3412154361523],[2428.288083881008],[1787.8045911666616],[0.8481197357177734],[2270.127009073893],[47.64584986201729],[14.570302050858505],[0.7809176974025754],[0.9419040343841134]
1,AirplaneF16.tiff,3,otsu,max,"[118, 174, 198]","[81, 135, 183]","[133, 185, 209]",956.660426,2531.457043,1846.387280,...,"[[133, 209, 209, 209, 209, 209, 209, 209, 209,...",[956.660425684024],[2531.457043226552],[1846.3872798698833],[0.7897562980651855],[1404.8118896484375],[37.48082029049574],[16.654621866318674],[0.7250285314302906],[0.960552463805854]
2,AirplaneF16.tiff,4,otsu,max,"[112, 144, 182, 204]","[0, 89, 160, 204]","[22, 117, 175, 203]",969.722077,2518.055294,1882.229683,...,"[[117, 203, 203, 255, 203, 203, 203, 203, 203,...",[969.7220766629255],[2518.0552944227993],[1882.229683035869],[0.7624030113220215],[1196.0068740844727],[34.583332316080714],[17.35346685088114],[0.7020996991642434],[0.9647617511564467]
3,AirplaneF16.tiff,5,otsu,max,"[58, 100, 134, 170, 192]","[73, 93, 122, 161, 203]","[117, 153, 173, 195, 204]",1002.558810,2598.019485,1910.525846,...,"[[117, 204, 204, 255, 204, 204, 204, 195, 204,...",[1002.5588096739694],[2598.0194853715097],[1910.5258458694248],[0.6703917980194092],[1248.2512664794922],[35.33059957712991],[17.167783455040507],[0.7331674054806244],[0.9696348432270101]
4,Lena.png,2,otsu,max,"[146, 208]","[79, 148]","[98, 140]",1003.657019,2393.080486,2155.639019,...,"[[140, 140, 140, 140, 140, 140, 140, 140, 140,...",[1003.6570185970401],[2393.0804857472363],[2155.6390194411747],[0.6338398456573486],[1832.102835337321],[42.803070396144726],[15.501305140436546],[0.6292027376463637],[0.8938510184214241]
5,Lena.png,3,otsu,max,"[129, 191, 223]","[56, 114, 160]","[80, 106, 148]",1067.482394,2587.880520,2250.903041,...,"[[148, 148, 148, 148, 148, 148, 148, 148, 148,...",[1067.482394152646],[2587.8805202304575],[2250.903041452398],[0.664649486541748],[1172.378079732259],[34.24000700543531],[17.440126711461378],[0.6958615855979055],[0.9275372775306022]
6,Lena.png,4,otsu,max,"[100, 142, 179, 199]","[36, 65, 116, 152]","[87, 116, 139, 171]",1088.982713,2629.162648,2276.494710,...,"[[139, 139, 139, 139, 139, 139, 139, 139, 139,...",[1088.9827125024788],[2629.1626483028413],[2276.4947095206544],[0.6828587055206299],[934.126651763916],[30.563485595787597],[18.426745976643623],[0.735853921224687],[0.9512625716812929]
7,Lena.png,5,otsu,max,"[115, 160, 193, 209, 212]","[40, 82, 109, 131, 175]","[70, 92, 113, 145, 172]",1113.074316,2701.260268,2315.559235,...,"[[145, 145, 145, 145, 145, 145, 145, 145, 145,...",[1113.074315763491],[2701.260267553662],[2315.559234793464],[0.5895423889160156],[618.7338676452637],[24.87436165302064],[20.215764722135226],[0.7690230733782099],[0.961285383952324]


# Save results

In [9]:
# pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_otsu_RGB_df.pkl', 'wb'))
# pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_otsu_RGB_dict.pkl', 'wb'))